# Import libraries

In [1]:
import pandas as pd
import numpy as np
from z3 import *
from itertools import combinations
import itertools
import matplotlib.pyplot as plt
import os
import json
import sys
sys.path.append("../scripts/")
from ControlCost import *
from ActualADM import *
from AttackCost import *
from LinearizedADM import *
import warnings
warnings.filterwarnings('ignore')

# Current and parent directories

In [2]:
current_directory = os.getcwd()
parent_directory = os.path.dirname(current_directory)

# Accessing Zone-Activity-Appliances Information

In [3]:
zones = pd.read_excel(str(parent_directory) + '/data/Aras-Information.xlsx', sheet_name='Zone-Info')
zone_volume = zones["Volume (cf)"].to_list()                  # Zones' volumes (cubic feet)

energy_appliances = pd.read_excel(str(parent_directory) + '/data/Aras-Information.xlsx', sheet_name='Appliances-Info').iloc[:,-1].to_numpy()
benign_activity_appliances_map = pd.read_excel(str(parent_directory) + '/data/Aras-Information.xlsx', sheet_name='Benign-Activity-Appliances-Map').iloc[:, 4:].to_numpy()

pp_co2 = zones["CO2 Emission by Occupant (cfm)"].to_list()    # CO2 Emission by Occupant (cfm)
pp_heat = zones["Heat Radiation by Occupant (W)"].to_list()   # Heat Radiation by Occupant (W)
load = zones["Heat Radiated by Appliances (W)"].to_list()     # Heat radiated by Appliances (W)

num_zones = len(zones)
num_timeslots = 1440

activities = pd.read_excel(str(parent_directory) + '/data/Aras-Information.xlsx', sheet_name='Activity-Info')
num_activities = len(activities)
activity_zone_map = dict()
for i in range(len(activities)):
    activity_zone_map[int(activities["Activity ID"][i])] = int(activities["Zone ID"][i])

activity_zone_temp_setpoint = [0]
activity_zone_co2_setpoint = [0] 
activity_pp_co2 = [0] + activities['CO2 Emission by Occupant (CFM)'].to_list()    # CO2 Emission by Occupant (cfm)
activity_pp_heat = [0] + activities["Heat Radiation by Occupant (W)"].to_list()   # Heat Radiation by Occupant (W)
activity_load = [0]# Heat radiated by Appliances (W)
activity_zone_volume = [0]

for i in range(1, num_activities + 1):
    activity_zone_temp_setpoint.append(zone_temp_setpoint[activity_zone_map[i]])
    activity_zone_co2_setpoint.append(zone_co2_setpoint[activity_zone_map[i]])
    activity_zone_volume.append(zone_volume[activity_zone_map[i]])
    activity_load.append(sum(benign_activity_appliances_map[i] * energy_appliances))
    
all_samples = list(itertools.permutations([1, 1, 0, 0, 0]))
all_samples += list(itertools.permutations([2, 0, 0, 0, 0]))

unique_samples = set()

for data in all_samples:    
    unique_samples.add(data)

# Convert list to a tuple

In [4]:
def to_tuple(a):
    try:
        return tuple(to_tuple(i) for i in a)
    except TypeError:
        return a

# Fixed parameters

In [5]:
CO2_FRESH_AIR = 400             # CO2 concentration (ppm) of fresh air
TEMP_FRESH_AIR = 91.4           # Temperature (33 degree F) of the fresh air
CP_AIR = 1.026                  # Specific heat of fresh air
DEF_TEMP_SUPPLY_AIR =  55.4     # Default temperature (degree fahrenheit) of supply air (13 degree celsius)

# Control cost calculation

In [6]:
def control_cost (zones, zone_occupant, zone_temp_setpoint, zone_volume, pp_co2, pp_heat, load, zone_co2_setpoint, control_time):
    '''
    PARAMETERS:
    zones: zone information
    zone_occupant: list of occupants in different zones
    zone_temp_setpoint: list of temperature (fahrenheit) setpoint of the different zones
    zone_volume: # Zones' volumes (cubic feet)
    pp_co2: CO2 Emission by Occupant (cfm)
    pp_heat: Heat Radiation by Occupant (W)
    load: Heat radiated by Appliances (W)
    zone_co2_setpoint: list of CO2 (ppm) setpoint of the corresponding zones
    control_time: time of control operation (in minute)
    energy_cost: cost per KWh (USD) at the control time
    '''
    num_zones = len(zones)
    # initializing z3 variables
    v_vent_air = [Real( 'v_vent_air_' + str(i)) for i in range(num_zones)]   # Air required for ventillation (CFM)
    v_temp_air = [Real( 'v_temp_air_' + str(i)) for i in range(num_zones)]   # Air required for cooling (CFM)
    v_mixed_air = [Real( 'v_mixed_air_' + str(i)) for i in range(num_zones)]
    v_fresh_air = [Real( 'v_fresh_air_' + str(i)) for i in range(num_zones)]
    v_return_air = [Real( 'v_return_air_' + str(i)) for i in range(num_zones)]
    zone_cost = [Real( 'zone_cost' + str(i)) for i in range(num_zones)] 
    
    temp_supply_air = [ Real( 'temp_supply_air_' + str(i)) for i in range(num_zones)]
    temp_mixed_air = [ Real( 'temp_mixed_air_' + str(i)) for i in range(num_zones)]
    co2_mixed_air = [ Real( 'co2_mixed_air_' + str(i)) for i in range(num_zones)]
    total_zone_cost = Real('total_zone_cost')

    s = Solver()
    
    for i in range(1, num_zones):
        ############### v_vent_air ###############################
        s.add(zone_occupant[i] * ((pp_co2[i] * 1000000) / zone_volume[i]) == 
                       (zone_co2_setpoint[i] - (( 1 - (v_vent_air[i]) /zone_volume[i]) * zone_co2_setpoint[i] +  
                                                (v_vent_air[i] * CO2_FRESH_AIR) /  zone_volume[i])))
        
        ############### v_temp_air ###############################
        if zone_occupant[i] > 0:
            s.add(v_temp_air[i] *  (zone_temp_setpoint[i] - DEF_TEMP_SUPPLY_AIR) * 0.3167 == zone_occupant[i] * pp_heat[i] + load[i]) 
        else:
            s.add(v_temp_air[i] *  (zone_temp_setpoint[i] - DEF_TEMP_SUPPLY_AIR) * 0.3167 == zone_occupant[i] * pp_heat[i]) 
    
        ############### v_mixed_air ###############################
        s.add(zone_occupant[i] * ((pp_co2[i] * 1000000) / zone_volume[i]) == 
              (zone_co2_setpoint[i] - (( 1 - ( v_mixed_air[i] ) / zone_volume[i]) * zone_co2_setpoint[i] + 
                                     ( v_mixed_air[i] * co2_mixed_air[i]) / zone_volume[i])))
    
        if zone_occupant[i] > 0:
            s.add(v_mixed_air[i] * (zone_temp_setpoint[i] - temp_supply_air[i]) * 0.3167 == zone_occupant[i] * pp_heat[i] + load[i])
        else:
            s.add(v_mixed_air[i] * (zone_temp_setpoint[i] - temp_supply_air[i]) * 0.3167 == zone_occupant[i] * pp_heat[i])

        s.add(v_mixed_air[i] == v_return_air[i] + v_fresh_air[i])
        s.add(co2_mixed_air[i] == zone_co2_setpoint[i] * (v_return_air[i] / v_mixed_air[i]) + CO2_FRESH_AIR * (v_fresh_air[i] / v_mixed_air[i]))
        s.add(temp_mixed_air[i] == zone_temp_setpoint[i] * (v_return_air[i] / v_mixed_air[i]) + TEMP_FRESH_AIR * (v_fresh_air[i] / v_mixed_air[i]))
    
        ############### temperature control algorithm ############
        s.add(Implies(v_vent_air[i] >= v_temp_air[i] , v_return_air[i] == 0))
        s.add(Implies(v_vent_air[i] < v_temp_air[i] ,  temp_supply_air[i] == 55.4))
    
        ############### other constraints ########################
        s.add(v_return_air[i] >= 0)
        s.add(temp_supply_air[i] >= 55.4)
        
        ############## cost constraint ###########################
        s.add(zone_cost[i] == v_mixed_air[i] * (temp_mixed_air[i] - DEF_TEMP_SUPPLY_AIR) * 0.3167 * (control_time / 60000))
    s.add(total_zone_cost == Sum(zone_cost[1:]))
    s.check()
    
    for i in range(1, num_zones):
        v_vent_air[i] = float(Fraction(str(s.model()[v_vent_air[i]])))
        v_temp_air[i] = float(Fraction(str(s.model()[v_temp_air[i]])))
    
        v_mixed_air[i] = float(Fraction(str(s.model()[v_mixed_air[i]])))
        temp_mixed_air[i] = float(Fraction(str(s.model()[temp_mixed_air[i]])))
    
        temp_supply_air[i] = float(Fraction(str(s.model()[temp_supply_air[i]])))
    
        co2_mixed_air[i] = float(Fraction(str(s.model()[co2_mixed_air[i]])))
        v_return_air[i] = float(Fraction(str(s.model()[v_return_air[i]])))
        v_fresh_air[i] = float(Fraction(str(s.model()[v_fresh_air[i]])))
        
        zone_cost[i] = float(Fraction(str(s.model()[zone_cost[i]])))
    total_zone_cost = float(Fraction(str(s.model()[total_zone_cost])))
        
    return total_zone_cost

dict_control_cost = dict()
for sample in unique_samples:
    zone_occupant = list(sample)    
    dict_control_cost[sample] = control_cost(zones, zone_occupant, zone_temp_setpoint, zone_volume, pp_co2, pp_heat, load, zone_co2_setpoint, control_time)

# Zone-wise appliances

In [7]:
zone_appliance = []
zone_appliance.append([1, 2]) #0
zone_appliance.append([11, 16, 17, 23, 27]) #1
zone_appliance.append([12, 13, 18, 22, 25, 26]) #2
zone_appliance.append([3, 4, 5, 6, 7, 8, 9, 10, 19, 24]) #3
zone_appliance.append([14, 15, 20, 21]) #4

# Zone-wise maximum cost for appliances

In [8]:
zone_cost = [0, 0, 0, 0, 0]
for i in range(len(zone_cost)):
    all_costs = []
    for j in range(len(zone_appliance[i])):
        all_costs.append(activity_load[zone_appliance[i][j]])
    zone_cost[i] =  max(all_costs)

# SHATTER Attack Scheduling Function

In [9]:
def attack_scheduling(list_time_min, list_time_max, start_time, final_time):
    prev_stay = 1
    prev_schedule = -1
    ultimate_cost = 0
    final_schedule = schedule = np.zeros((final_time - start_time))
    num_timeslots = interval = 10

    for init_time in range(start_time, final_time, interval):
        cost = [Int( 'cost_' + str(i)) for i in range(NUM_ZONES)]
        zones = [Int( 'zones_' + str(i)) for i in range(NUM_ZONES)]

        schedule = [Int( 'schedule_' + str(i)) for i in range(init_time, init_time + interval)]
        stay = [Int( 'stay_' + str(i)) for i in range(num_timeslots)]
        slot_cost = [Int( 'slot_cost_' + str(i)) for i in range(num_timeslots)]

        total_cost = Int('total_cost')

        o = Optimize()
        o.add(cost[0] == 0)
        o.add(cost[1] == 1)
        o.add(cost[2] == 2)
        o.add(cost[3] == 4)
        o.add(cost[4] == 3)


        ############################################################################
        ################## schedule should be withing a valid zone #################
        ############################################################################
        for t in range(len(schedule)):
            or_constraints = []
            for z in range(NUM_ZONES):
                or_constraints.append(schedule[t] == z)
            o.add(Or(or_constraints))

        ###############################################################################################################
        ################## if zone stay threshdold in current time is 0, do not schedule to that zone #################
        ###############################################################################################################

        # base case
        for z in range(NUM_ZONES):
            if list_time_min[z][init_time] == [] or (len(list_time_max[z][init_time]) == 1 and list_time_max[z][init_time][0] == 0):
                o.add(Implies(schedule[0] != prev_schedule, schedule[0] != z))

        for t in range(1, len(schedule)):
            for z in range(NUM_ZONES):
                if list_time_min[z][init_time + t] == [] or (len(list_time_max[z][init_time + t]) == 1 and list_time_max[z][init_time + t][0] == 0):
                    o.add(Implies(schedule[t] != schedule[t - 1], schedule[t] != z))

        #######################################################################################################################
        ############################################ constraints of stay ######################################################
        #######################################################################################################################
        ######## base case for time 0 ############
        if init_time == 0:
            o.add(stay[0] == 1)
        else:
            o.add(Implies(schedule[0] == prev_schedule, stay[0] == prev_stay + 1))
            o.add(Implies(Not(schedule[0] == prev_schedule), stay[0] == 1))

        for t in range(1, len(schedule)):
            continue_staying = (schedule[t] == schedule[t - 1])
            increment_stay = (stay[t] == stay[t - 1] + 1)
            reset_stay = (stay[t] == 1)

            o.add(Implies(continue_staying, increment_stay))
            o.add(Implies(Not(continue_staying), reset_stay))

        #######################################################################################################################
        ############ move to a zone different that previous timeslot if stay > max threshold in previous timeslot #############
        #######################################################################################################################
        ######## base case for time 0 ############
        o.add(Implies(prev_stay == max(list_time_max[prev_schedule][init_time - prev_stay], default=0), schedule[0] != prev_schedule))

        for t in range(1, len(schedule)):
            max_stay_threshold = 0
            for z in range(NUM_ZONES):
                for p_t in range(1, init_time + len(schedule)):
                    continue_staying = (schedule[t] == schedule[t - 1])
                    o.add(Implies(And(schedule[t - 1] == z, stay[t - 1] == p_t, p_t == max(list_time_max[z][init_time + t - p_t], default=0)), Not(continue_staying)))

        #######################################################################################################################
        # must stay in the zone same as the previous timeslot if stay < max && stay is in previous timeslot is out of cluster #
        #######################################################################################################################
        ######## base case for time 0 ############
        ranges_stay_constraints = []
        for k in range(len(list_time_min[prev_schedule][init_time - prev_stay])):
            ranges_stay_constraints.append(And(prev_stay >= list_time_min[prev_schedule][init_time - prev_stay][k], prev_stay <= list_time_max[prev_schedule][init_time - prev_stay][k]))            

        if init_time != 0:
            o.add(Implies(Not(Or(ranges_stay_constraints)), schedule[0] == prev_schedule))

        for t in range(1, len(schedule)):
            for z in range(NUM_ZONES):
                for p_t in range(1, init_time + t + 1):
                    ranges_stay_constraints = []
                    for k in range(len(list_time_min[z][init_time + t - p_t])):
                        ranges_stay_constraints.append(And(p_t >= list_time_min[z][init_time + t - p_t][k], p_t <= list_time_max[z][init_time + t - p_t][k]))            

                    continue_staying = (schedule[t] == schedule[t - 1])
                    o.add(Implies(And(schedule[t - 1] == z, stay[t - 1] == p_t, Not(Or(ranges_stay_constraints))), continue_staying))

        for t in range(len(schedule)):
            for z in range(NUM_ZONES):
                o.add(Implies(schedule[t] == z, slot_cost[t] == cost[z]))

        o.add(total_cost == Sum(slot_cost))

        #o.add(schedule[0] == 3)

        o.maximize(total_cost)

        o.check()
        
        if o.check() == unsat:
            print('unsat', init_time)
            return final_schedule, init_time - prev_stay
            
        
        #print(init_time, o.model()[total_cost])
        #ultimate_cost += int(str(o.model()[total_cost]))

        #print("Ulimate Cost", init_time, ultimate_cost)

        #print("num_timeslot", num_timeslots, "optimal_cost", o.model()[total_cost])

        for t in range(num_timeslots):
            #print("schedule_" + str(init_time + t), o.model()[schedule[t]])
            final_schedule[init_time + t] = int(str(o.model()[schedule[t]]))

        #for t in range(num_timeslots):
        #    print("stay_" + str(init_time + t), o.model()[stay[t]])

        prev_schedule = int(str(o.model()[schedule[-1]]))
        prev_stay = int(str(o.model()[stay[-1]]))
    return final_schedule

In [10]:
linearized_shchecker_adm = pd.DataFrame()

for house_name in ['A']:
    for adm_algo in ["DBSCAN", "K-Means"]:
        for occ_id in ['1', '2']:
            for knowledge in ["Full", "Partial"]:  
                print(house_name, adm_algo, occ_id, knowledge)

                dataframe = pd.read_csv(str(parent_directory) + '\data\\modified\\' + 'Modified-Dataframe_House-' + str(house_name) + '.csv')

                cleaned_dataframe_occ = pd.read_csv(str(parent_directory) + '\data\\cleaned\\' + 'Cleaned-Dataframe_House-' + str(house_name) + '_Occupant-' + str(occ_id) + '.csv')

                processed_dataframe = pd.read_csv(str(parent_directory) + '\data\\processed\\' + 'Processed-Dataframe_House-' + str(house_name) + '.csv')

                actual_adm_occ = ActualADM(adm_algo, cleaned_dataframe_occ, knowledge, house_name, occ_id, num_timeslots, num_zones)
                list_time_min_occ, list_time_max_occ = actual_adm_occ.noise_augmented_range_calculation()

                attack_schedule = attack_scheduling(list_time_min_occ, list_time_max_occ, 0, 1440)

A DBSCAN 1 Full
unsat 1130
A DBSCAN 1 Partial
A DBSCAN 2 Full
unsat 1430
A DBSCAN 2 Partial
unsat 80
A K-Means 1 Full
A K-Means 1 Partial
unsat 330
A K-Means 2 Full
A K-Means 2 Partial
B DBSCAN 1 Full
unsat 80
B DBSCAN 1 Partial
unsat 0
B DBSCAN 2 Full
unsat 0
B DBSCAN 2 Partial
unsat 0
B K-Means 1 Full
B K-Means 1 Partial
B K-Means 2 Full
B K-Means 2 Partial
